In [ ]:
!pip install --no-cache-dir -r requirements.txt
!python Create_synth_data.py

^C



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pyspark

logfile = "logfiles.log"
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LogFileAnalysis") \
    .getOrCreate()


In [13]:
help(pl.scan_csv)

Help on function scan_csv in module polars.io.csv.functions:

scan_csv(source: 'str | Path | IO[str] | IO[bytes] | bytes | list[str] | list[Path] | list[IO[str]] | list[IO[bytes]] | list[bytes]', *, has_header: 'bool' = True, separator: 'str' = ',', comment_prefix: 'str | None' = None, quote_char: 'str | None' = '"', skip_rows: 'int' = 0, skip_lines: 'int' = 0, schema: 'SchemaDict | None' = None, schema_overrides: 'SchemaDict | Sequence[PolarsDataType] | None' = None, null_values: 'str | Sequence[str] | dict[str, str] | None' = None, missing_utf8_is_empty_string: 'bool' = False, ignore_errors: 'bool' = False, cache: 'bool' = True, with_column_names: 'Callable[[list[str]], list[str]] | None' = None, infer_schema: 'bool' = True, infer_schema_length: 'int | None' = 100, n_rows: 'int | None' = None, encoding: 'CsvEncoding' = 'utf8', low_memory: 'bool' = False, rechunk: 'bool' = False, skip_rows_after_header: 'int' = 0, row_index_name: 'str | None' = None, row_index_offset: 'int' = 0, try_p

In [339]:
# improt polars
import polars as pl
# read the first 50 lines of the log file into a DataFrame
df = pl.scan_csv(logfile, has_header=False, separator = '~')
# print the DataFrame to the console
df.collect()
# apply regex to the DataFrame
df.with_columns(
    pl.col("column_1").str.extract(r"((\d{1,3}\.){3}\d{1,3})").alias("IP"),
    pl.col("column_1").str.extract(r"\[(\d{1,2}/\w{3}/\d{4})").alias("date"),
    pl.col("column_1").str.extract(r'"(\w*)').alias("method"),
    pl.col("column_1").str.extract(r'"\w* (/\w*){1,3} HTTP/1.0').alias("user_type"),
    pl.col("column_1").str.extract(r'1.0" (\d{3})').alias("Status"),
    pl.col("column_1").str.extract(r'1.0" \d{3} (\d{1,4})').alias("port"),
    pl.col("column_1").str.extract(r'1\.0"\s\d{3}\s\d+\s"([^"]*)"').alias("site"),
    pl.col("column_1").str.extract(r":(\d{2}):\d{2}:\d{2}").cast(pl.Int32).alias("hour"),
    pl.col("column_1").str.extract(r":\d{2}:(\d{2}):\d{2}").cast(pl.Int32).alias("minute"),
    pl.col("column_1").str.extract(r":\d{2}:\d{2}:(\d{2})").cast(pl.Int32).alias("second"),
    pl.col("column_1").str.extract(r"\+(\d{2})\d{2}").cast(pl.Int32).alias("timezone_offset"),
).with_columns(
    ((pl.col("hour") - pl.col("timezone_offset")) % 24).alias("adjusted_hour"),
    pl.col("user_type").str.strip_prefix("/").alias("user_type")
).with_columns(
    pl.concat_str([
        pl.col("adjusted_hour").cast(pl.Utf8).str.zfill(2),
        pl.col("minute").cast(pl.Utf8).str.zfill(2),
        pl.col("second").cast(pl.Utf8).str.zfill(2),
    ], separator=":").alias("adjusted_time_string")
).select(
    "IP", "date", "adjusted_time_string", "method", "user_type", "Status", "port",
).collect()

df.with_columns(pl.col("column_1").str.slice(70)).collect()
#df.with_columns(pl.col("column_1").str.extract(r'"\w* (/\w*[/, ]\w*[/, ]\w*)').alias("Method")).collect()


column_1
str
""" 404 5021 ""http://www.beard.co…"
"""404 4997 ""http://www.beard.com…"
""""" 404 5042 ""http://www.beard.c…"
""" 500 5037 ""http://www.beard.co…"
"""P/1.0"" 502 5095 ""-"" ""Mozilla/5…"
…
""" 5026 ""-"" ""Mozilla/5.0 (Macint…"
""".0"" 303 5052 ""http://www.beard…"
""".0"" 403 5096 ""-"" ""Mozilla/5.0 …"


In [338]:
# find unique values starting from the end of r'"(\w*)' and ending with /1.0
df.with_columns(pl.col("column_1").str.extract(r'1\.0"\s\d{3}\s\d+\s"([^"]*)"').alias("site")).collect()


column_1,site
str,str
"""73.114.209.133 - - [30/Aug/201…","""http://www.beard.com/categorie…"
"""223.114.129.136 - - [01/Oct/20…","""http://www.beard.com/categorie…"
"""136.219.6.68 - - [30/Jan/2018:…","""http://www.beard.com/categorie…"
"""12.234.4.43 - - [20/Mar/2019:0…","""http://www.beard.com/categorie…"
"""199.150.229.44 - - [30/Sep/201…","""-"""
…,…
"""185.129.61.42 - - [18/Aug/2019…","""-"""
"""176.31.181.203 - - [31/Jul/201…","""http://www.beard.com/categorie…"
"""49.247.162.253 - - [15/Aug/201…","""-"""


In [3]:
logs = spark.read.text(logfile)

In [15]:
from pyspark.sql.functions import regexp_extract, col

# take the first 50 lines and pu them in a new dataframe
logs_tiny = logs.limit(50)
#logs_tiny.show(truncate=False)
#logs_tiny.select(regexp_extract('value', r'(\d+\.\d+\.\d+\.\d+)', 1).alias('ip')).show(truncate=False)
logs_tiny.select(regexp_extract('value', r'[(\d{1,2}/\n{3}/\d{4})]', 1).alias('date')).show()

Py4JJavaError: An error occurred while calling o100.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 15) (DESKTOP-6O57JT6 executor driver): org.apache.spark.SparkRuntimeException: [INVALID_PARAMETER_VALUE.REGEX_GROUP_INDEX] The value of parameter(s) `idx` in `regexp_extract` is invalid: Expects group index between 0 and 0, but got 1.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.invalidRegexGroupIndexError(QueryExecutionErrors.scala:357)
	at org.apache.spark.sql.catalyst.expressions.RegExpExtractBase$.checkGroupIndex(regexpExpressions.scala:735)
	at org.apache.spark.sql.catalyst.expressions.RegExpExtractBase.checkGroupIndex(regexpExpressions.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.j